# Protecting against Cross-Site Scripting

This notebook provides an example to showcase the methods we use to protect against Cross-Site Scripting (XSS).
In particular, to protect against it we escape some symbols in the JSON output and we add some extra headers which further ensure that the browser won't misidentify the content.

* **JSON serialiser.** We escape any HTML symbols on the output, using their unicode sequences instead.
* **Extra headers.** On every response, we set the `X-Content-Type-Options: nosniff;` header, which ensures that the browser won't try to guess the `Content-Type` from the content.

## Setup

Before showing a couple examples on how the output is modified to protect against XSS attacks, we will setup the environment.

### Build engine image

To make sure we are running the latest version of the engine, we will build a docker image from the current code.
Note that this requires a valid JDK installation.

In [1]:
!cd ../../../engine && make build_image

../proto/seldon_deployment.proto -> src/main/proto/seldon_deployment.proto
../proto/prediction.proto -> src/main/proto/prediction.proto
cp -vr ../proto/k8s/k8s.io src/main/proto
../proto/k8s/k8s.io -> src/main/proto/k8s.io
../proto/k8s/k8s.io/apis -> src/main/proto/k8s.io/apis
../proto/k8s/k8s.io/apis/meta -> src/main/proto/k8s.io/apis/meta
../proto/k8s/k8s.io/apis/meta/v1 -> src/main/proto/k8s.io/apis/meta/v1
../proto/k8s/k8s.io/api -> src/main/proto/k8s.io/api
../proto/k8s/k8s.io/api/core -> src/main/proto/k8s.io/api/core
../proto/k8s/k8s.io/api/core/v1 -> src/main/proto/k8s.io/api/core/v1
../proto/k8s/k8s.io/api/core/v1/generated.proto -> src/main/proto/k8s.io/api/core/v1/generated.proto
../proto/k8s/k8s.io/api/core/v1/generated.protobak -> src/main/proto/k8s.io/api/core/v1/generated.protobak
../proto/k8s/k8s.io/api/autoscaling -> src/main/proto/k8s.io/api/autoscaling
../proto/k8s/k8s.io/api/autoscaling/v2beta1 -> src/main/proto/k8s.io/api/autoscaling/v2beta1
../proto/k8s/k8s.io/api

15:28:18.361 [main] INFO org.springframework.boot.test.context.SpringBootTestContextBootstrapper - Neither @ContextConfiguration nor @ContextHierarchy found for test class [io.seldon.engine.predictors.RandomABTestUnitTest], using SpringBootContextLoader
15:28:18.385 [main] DEBUG org.springframework.test.context.support.AbstractContextLoader - Did not detect default resource location for test class [io.seldon.engine.predictors.RandomABTestUnitTest]: class path resource [io/seldon/engine/predictors/RandomABTestUnitTest-context.xml] does not exist
15:28:18.387 [main] DEBUG org.springframework.test.context.support.AbstractContextLoader - Did not detect default resource location for test class [io.seldon.engine.predictors.RandomABTestUnitTest]: class path resource [io/seldon/engine/predictors/RandomABTestUnitTestContext.groovy] does not exist
15:28:18.387 [main] INFO org.springframework.test.context.support.AbstractContextLoader - Could not detect default resource locations for test class [

15:28:18.676 [main] DEBUG org.springframework.core.io.support.PathMatchingResourcePatternResolver - Resolved classpath location [io/seldon/engine/predictors/] to resources [URL [file:/Users/kaseyo/Seldon/seldon-core-mirror1/engine/target/test-classes/io/seldon/engine/predictors/], URL [file:/Users/kaseyo/Seldon/seldon-core-mirror1/engine/target/classes/io/seldon/engine/predictors/]]
15:28:18.677 [main] DEBUG org.springframework.core.io.support.PathMatchingResourcePatternResolver - Looking for matching resources in directory tree [/Users/kaseyo/Seldon/seldon-core-mirror1/engine/target/test-classes/io/seldon/engine/predictors]
15:28:18.677 [main] DEBUG org.springframework.core.io.support.PathMatchingResourcePatternResolver - Searching directory [/Users/kaseyo/Seldon/seldon-core-mirror1/engine/target/test-classes/io/seldon/engine/predictors] for files matching pattern [/Users/kaseyo/Seldon/seldon-core-mirror1/engine/target/test-classes/io/seldon/engine/predictors/*.class]
15:28:18.694 [ma

15:28:19.131 [main] INFO org.springframework.boot.test.context.SpringBootTestContextBootstrapper - Using TestExecutionListeners: [org.springframework.test.context.web.ServletTestExecutionListener@32bb0072, org.springframework.test.context.support.DirtiesContextBeforeModesTestExecutionListener@467233e4, org.springframework.boot.test.mock.mockito.MockitoTestExecutionListener@427a12b6, org.springframework.boot.test.autoconfigure.SpringBootDependencyInjectionTestExecutionListener@6025d790, org.springframework.test.context.support.DirtiesContextTestExecutionListener@af7e376, org.springframework.test.context.transaction.TransactionalTestExecutionListener@5dcd0cdf, org.springframework.test.context.jdbc.SqlScriptsTestExecutionListener@4fb04a72, org.springframework.boot.test.mock.mockito.ResetMocksTestExecutionListener@1e79d43, org.springframework.boot.test.autoconfigure.restdocs.RestDocsTestExecutionListener@343e225a, org.springframework.boot.test.autoconfigure.web.client.MockRestServiceServer

15:28:19.194 [main] DEBUG org.springframework.test.context.junit4.SpringJUnit4ClassRunner - SpringJUnit4ClassRunner constructor called with [class io.seldon.engine.api.rest.TestRandomABTest]
15:28:19.195 [main] DEBUG org.springframework.test.context.BootstrapUtils - Instantiating CacheAwareContextLoaderDelegate from class [org.springframework.test.context.cache.DefaultCacheAwareContextLoaderDelegate]
15:28:19.196 [main] DEBUG org.springframework.test.context.BootstrapUtils - Instantiating BootstrapContext using constructor [public org.springframework.test.context.support.DefaultBootstrapContext(java.lang.Class,org.springframework.test.context.CacheAwareContextLoaderDelegate)]
15:28:19.201 [main] DEBUG org.springframework.test.context.BootstrapUtils - Instantiating TestContextBootstrapper for test class [io.seldon.engine.api.rest.TestRandomABTest] from class [org.springframework.boot.test.context.SpringBootTestContextBootstrapper]
15:28:19.203 [main] INFO org.springframework.boot.test.c

...

2019-09-27 15:28:41.400  INFO 76901 --- [askExecutor-137] i.s.e.service.InternalPredictionService  : Calling grpc for transform-input
{
  "meta": {
    "puid": "u0b2e4810ucrrk8238u7ki04kg",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
    },
    "metrics": []
  }
}
{
  "meta": {
    "puid": "u0b2e4810ucrrk8238u7ki04kg",
    "tags": {
    },
    "routing": {
      "abtest": 0
    },
    "requestPath": {
      "abtest": "",
      "model1": "seldonio/model1:0.6"
    },
    "metrics": [{
      "key": "mycounter",
      "type": "COUNTER",
      "value": 1.0,
      "tags": {
        "mytag1": "mytagval1"
      }
    }, {
      "key": "mygauge",
      "type": "GAUGE",
      "value": 22.0,
      "tags": {
      }
    }, {
      "key": "mytimer",
      "type": "TIMER",
      "value": 1.0,
      "tags": {
      }
    }]
  },
  "data": {
    "names": [],
    "ndarray": [[1.0, 2.0]]
  }
}
{
  "meta": {
    "puid": "u0b2e4810ucrrk8238u7ki04kg",
    "tags": {
    },
    "routing

2019-09-27 15:28:41.596  INFO 76901 --- [askExecutor-157] i.s.e.service.InternalPredictionService  : Calling grpc for transform-input
{
  "meta": {
    "puid": "af5t3g6gd2roj1jquqnlg1si8s",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
    },
    "metrics": []
  }
}
{
  "meta": {
    "puid": "af5t3g6gd2roj1jquqnlg1si8s",
    "tags": {
    },
    "routing": {
      "abtest": 0
    },
    "requestPath": {
      "abtest": "",
      "model1": "seldonio/model1:0.6"
    },
    "metrics": [{
      "key": "mycounter",
      "type": "COUNTER",
      "value": 1.0,
      "tags": {
        "mytag1": "mytagval1"
      }
    }, {
      "key": "mygauge",
      "type": "GAUGE",
      "value": 22.0,
      "tags": {
      }
    }, {
      "key": "mytimer",
      "type": "TIMER",
      "value": 1.0,
      "tags": {
      }
    }]
  },
  "data": {
    "names": [],
    "ndarray": [[1.0, 2.0]]
  }
}
{
  "meta": {
    "puid": "af5t3g6gd2roj1jquqnlg1si8s",
    "tags": {
    },
    "routing

2019-09-27 15:28:41.716  INFO 76901 --- [askExecutor-173] i.s.e.service.InternalPredictionService  : Calling grpc for transform-input
{
  "meta": {
    "puid": "t85f57osikl4ptompvd1hib39d",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
    },
    "metrics": []
  }
}
{
  "meta": {
    "puid": "t85f57osikl4ptompvd1hib39d",
    "tags": {
    },
    "routing": {
      "abtest": 0
    },
    "requestPath": {
      "abtest": "",
      "model1": "seldonio/model1:0.6"
    },
    "metrics": [{
      "key": "mycounter",
      "type": "COUNTER",
      "value": 1.0,
      "tags": {
        "mytag1": "mytagval1"
      }
    }, {
      "key": "mygauge",
      "type": "GAUGE",
      "value": 22.0,
      "tags": {
      }
    }, {
      "key": "mytimer",
      "type": "TIMER",
      "value": 1.0,
      "tags": {
      }
    }]
  },
  "data": {
    "names": [],
    "ndarray": [[1.0, 2.0]]
  }
}
{
  "meta": {
    "puid": "t85f57osikl4ptompvd1hib39d",
    "tags": {
    },
    "routing

2019-09-27 15:28:41.808  INFO 76901 --- [askExecutor-195] i.s.e.service.InternalPredictionService  : Calling grpc for transform-input
{
  "meta": {
    "puid": "1gfvlqqtuqe91me756bdnhkcue",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
    },
    "metrics": []
  }
}
{
  "meta": {
    "puid": "1gfvlqqtuqe91me756bdnhkcue",
    "tags": {
    },
    "routing": {
      "abtest": 1
    },
    "requestPath": {
      "abtest": "",
      "model2": "seldonio/model2:0.6"
    },
    "metrics": [{
      "key": "mycounter",
      "type": "COUNTER",
      "value": 1.0,
      "tags": {
        "mytag1": "mytagval1"
      }
    }, {
      "key": "mygauge",
      "type": "GAUGE",
      "value": 22.0,
      "tags": {
      }
    }, {
      "key": "mytimer",
      "type": "TIMER",
      "value": 1.0,
      "tags": {
      }
    }]
  },
  "data": {
    "names": [],
    "ndarray": [[1.0, 2.0]]
  }
}
{
  "meta": {
    "puid": "1gfvlqqtuqe91me756bdnhkcue",
    "tags": {
    },
    "routing

[INFO] 
[INFO] --- license-maven-plugin:1.13:add-third-party (default) @ seldon-engine ---
[INFO] Load missing file /Users/kaseyo/Seldon/seldon-core-mirror1/engine/src/license/THIRD-PARTY.properties
[INFO] Missing file /Users/kaseyo/Seldon/seldon-core-mirror1/engine/src/license/THIRD-PARTY.properties is up-to-date.
[INFO] Writing third-party file to /Users/kaseyo/Seldon/seldon-core-mirror1/engine/target/generated-sources/license/LICENSES_THIRD_PARTY
[INFO] Will attach third party file from /Users/kaseyo/Seldon/seldon-core-mirror1/engine/src/license/THIRD-PARTY.properties
[INFO] 
[INFO] --- license-maven-plugin:1.13:download-licenses (default) @ seldon-engine ---
[WARNING] POM for dependency org.mockito:mockito-core has a license URL that returns file not found: http://github.com/mockito/mockito/blob/master/LICENSE
[WARNING] POM for dependency com.microsoft.azure:adal4j has an invalid license URL: null
[WARNING] POM for dependency jline:jline has an invalid license URL: LICENSE.txt
[WAR

### Create k8s cluster

Firstly, we will create a cluster using [kind](https://kind.sigs.k8s.io).

In [2]:
!kind create cluster
!export KUBECONFIG="$(kind get kubeconfig-path --name=kind)"

Creating cluster "kind" ...
 ✓ Ensuring node image (kindest/node:v1.16.3) 🖼
 ✓ Preparing nodes 📦7l
 ✓ Writing configuration 📜7l
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Have a question, bug, or feature request? Let us know! https://kind.sigs.k8s.io/#community 🙂
`kind get kubeconfig-path` is deprecated!

KIND will export and merge kubeconfig like kops, minikube, etc.
This command is now unnecessary and will be removed in a future release.

For more info see: https://github.com/kubernetes-sigs/kind/issues/1060
See also the output of `kind create cluster`



## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

Next, **before installing `seldon-core`**, we load the engine image we have just built above into the cluster.

In [10]:
!kind load docker-image seldonio/engine:$(cat ../../../engine/target/version.txt)

We can now install `seldon-core` on the new cluster, making sure that it uses the engine image local to the nodes.

In [15]:
!helm install seldon-core \
    ../../../helm-charts/seldon-core-operator \
    --namespace seldon-system \
    --set engine.image.pullPolicy=Never \
    --set usagemetrics.enabled=true \
    --set ambassador.enabled=true
!kubectl rollout status statefulset.apps/seldon-operator-controller-manager -n seldon-system

NAME:   seldon-core
LAST DEPLOYED: Fri Sep 27 15:35:06 2019
NAMESPACE: seldon-system
STATUS: DEPLOYED

RESOURCES:
==> v1/ClusterRole
NAME                          AGE
seldon-operator-manager-role  2s

==> v1/ClusterRoleBinding
NAME                                 AGE
seldon-operator-manager-rolebinding  2s

==> v1/ConfigMap
NAME           DATA  AGE
seldon-config  1     2s

==> v1/Pod(related)
NAME                                  READY  STATUS             RESTARTS  AGE
seldon-operator-controller-manager-0  0/1    ContainerCreating  0         1s

==> v1/Secret
NAME                                   TYPE    DATA  AGE
seldon-operator-webhook-server-secret  Opaque  0     2s

==> v1/Service
NAME                                        TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)  AGE
seldon-operator-controller-manager-service  ClusterIP  10.100.117.162  <none>       443/TCP  2s
webhook-server-service                      ClusterIP  10.103.59.142   <none>       443/TCP  2s

==> v1/ServiceA

### Dummy Model

To test how `seldon-core` processes the output to prevent XSS attacks we will use a dummy model which just replies with whatever input we send.
The code for this model can be seen below.

In [17]:
!pygmentize ./XSSModel.py

class XSSModel(object):
    """
    Dummy model which just returns its input back.
    """

    def predict(self, X, feature_names):
        return X


Firstly, we will build an appropiate image using `s2i`.
The name of this image will be `xss-model:0.1`.

In [18]:
!make build_image

s2i build . seldonio/seldon-core-s2i-python3:1.4.0-dev_20201023092145 xss-model:0.1
error: Unable to load docker config: json: cannot unmarshal string into Go value of type docker.dockerConfig
---> Installing application source...
Build completed successfully


We are now ready to spin up a service running our model.
Note that before, we need to load the image into our `kind` cluster.

In [19]:
!kind load docker-image xss-model:0.1

In [20]:
!pygmentize ./xss-example.json

{
  "apiVersion": "machinelearning.seldon.io/v1alpha2",
  "kind": "SeldonDeployment",
  "metadata": {
    "labels": {
      "app": "seldon"
    },
    "name": "xss-example"
  },
  "spec": {
    "name": "xss-example",
    "predictors": [
      {
        "componentSpecs": [
          {
            "spec": {
              "containers": [
                {
                  "image": "xss-model:0.1",
                  "imagePullPolicy": "IfNotPresent",
                  "name": "xss-model"
                }
              ]
            }
          }
        ],
        "graph": {
          "children": [],
          "name": "xss-model",
          "endpoint": {
            "type": "REST"
          },
          "type": "MODEL"
        },
        "name": "xss-model",
        "replicas": 1
      }
    ]
  }
}


In [21]:
!kubectl apply -f ./xss-example.json

seldondeployment.machinelearning.seldon.io/xss-example created


To visualise what the model does and verify that everything is working we can make an example request using `curl`.
Note that, on the request we are passing a string field as `{"strData": "hello world"}`.
On the output, we receive the same field after being returned as-is by `XSSModel`.

In [25]:
!curl \
    -X POST \
    -H 'Content-Type: application/json' \
    -d '{"strData": "hello world"}' \
    localhost:8003/seldon/default/xss-example/api/v0.1/predictions

{
  "meta": {
    "puid": "rn6a2h894ljkqm7rdnn50o2fvb",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "xss-model": "xss-model:0.1"
    },
    "metrics": []
  },
  "strData": "hello world"
}

## Checking the response

### JSON serialiser

To showcase the escaping of HTML characters in the JSON output, we will submit a HTML payload in our request.
Note that the output uses the corresponding unicode value, instead of the sensitive character.
This helps to avoid undesired behaviour when the output could be mis-interpreted as HTML.

In [26]:
!curl \
    -X POST \
    -H 'Content-Type: application/json' \
    -d '{"strData": "<div class=\"box\">This is a div</div>"}' \
    localhost:8003/seldon/default/xss-example/api/v0.1/predictions

{
  "meta": {
    "puid": "9bp9uqjkduef1qvn3dnh69a566",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "xss-model": "xss-model:0.1"
    },
    "metrics": []
  },
  "strData": "\u003cdiv class\u003d\"box\"\u003eThis is a div\u003c/div\u003e"
}

We can also verify that the output for anything else remains untouched.

In [27]:
!curl \
    -X POST \
    -H 'Content-Type: application/json' \
    -d '{"strData": "Not HTML!"}' \
    localhost:8003/seldon/default/xss-example/api/v0.1/predictions

{
  "meta": {
    "puid": "8el2h1vrc9vjq9b05hicblg2ha",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "xss-model": "xss-model:0.1"
    },
    "metrics": []
  },
  "strData": "Not HTML!"
}

In [28]:
!curl \
    -X POST \
    -H 'Content-Type: application/json' \
    -d '{"data": {"ndarray": [0, 1, 2, 3, 4]}}' \
    localhost:8003/seldon/default/xss-example/api/v0.1/predictions

{
  "meta": {
    "puid": "nf9ss4h3r8s6umcjtfe66rf8ad",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "xss-model": "xss-model:0.1"
    },
    "metrics": []
  },
  "data": {
    "names": [],
    "ndarray": [0.0, 1.0, 2.0, 3.0, 4.0]
  }
}

### Extra headers

Similarly, we can show the response headers, to see that the `X-Content-Type-Options` header is included in the response.
This header will avoid the browser trying to infer the content type and trusting the already sent `Content-Type` header instead.

In [29]:
!curl \
    -X POST \
    -sD - -o /dev/null \
    -H 'Content-Type: application/json' \
    -d '{"strData": "<div class=\"box\">This is a div</div>"}' \
    localhost:8003/seldon/default/xss-example/api/v0.1/predictions

### Using `SeldonClient`

To verify everything else still works as expected, we can use the `SeldonClient` to check that the responses are still interpreted correctly.

In [31]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(deployment_name='xss-example', namespace="default")

In [32]:
r = sc.predict(gateway='ambassador', str_data="<div class=\"box\">This is a div</div>")
print(r)

Success:True message:
Request:
strData: "<div class=\"box\">This is a div</div>"

Response:
meta {
  puid: "dn7k9fjim3n167bf9memfa8sn"
  requestPath {
    key: "xss-model"
    value: "xss-model:0.1"
  }
}
strData: "<div class=\"box\">This is a div</div>"



As we can see above, even though the output is now escaped, `SeldonClient` parses the `utf8`-encoded elements into their actual characters.
Therefore, using the client, the change is transparent.